In [ ]:
## https://medium.com/@dilip.voleti/classification-using-word2vec-b1d79d375381
#https://www.kaggle.com/code/ratthachat/workshop-text-classification-quora/notebook?fbclid=IwAR3G-ObwMSx5YXAEZDano78BX0sTpedM2lT0BfQdQ8rVhx-mlYCR-_wbkFs

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 23.9 MB/s eta 0:00:00


In [4]:
# Read in the data and clean up column names
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from pythainlp.tokenize import word_tokenize

In [5]:
# df = pd.read_csv('/content/drive/MyDrive/Thai NLP sentiment analysis/Dataset/Train01_Clear.csv',)
messages = pd.read_csv('/content/drive/MyDrive/Thai NLP sentiment analysis/Dataset/Train01_Clear.csv')
messages

,Unnamed: 0,Class,Text
0,0,neutral,ประเทศ เรา ผลิต และ ส่งออก ยาสูบ เยอะ สุด ใน โ...
1,1,neutral,คะ
2,2,negative,อิ เหี้ย ออม ทำ กู อยาก กิน เอ็ม เค
3,4,neutral,สวัสดี วัน พุธ แนน อะไร นะ
4,5,neutral,ก้อน ขอบพระคุณ มาก นะ คร้าบ ที่ มาหา ก้อน และ ...
...,...,...,...
23286,24058,negative,แม่ ง ควาย ล้วน นน
23287,24059,negative,ดอย สุเทพ น้อง ง ไป หมด แล้ว ววว
23288,24060,negative,ค่า ชุด อาจจะ แพง กว่า ส่วนลด
23289,24061,neutral,รัฐ ต้องการ แค่ ภาษี ครับ


In [ ]:
# # ตัดคำและลบ URL ทิ้ง (ใช้ loop เพื่อให้ code ง่ายแก่การอ่าน)
# articles = []
# for text in messages['Text']:
#     ar = []
#     for token in text.split(" "): # แบ่งคำด้วยเครื่องหมาย "|"
#         if ("www" in token.lower()):
#             if(len(ar)>0):
#                 articles.append(ar)
#             ar =[]
#         else:
#             ar.append(token)
#     articles.append(ar)

# articles[0]

In [ ]:
# messages['text_clean'] = articles
# messages

In [7]:
messages['Text'][0]

'ประเทศ เรา ผลิต และ ส่งออก ยาสูบ เยอะ สุด ใน โลก จิง ป่าว คับ'

In [ ]:
# from pythainlp.corpus import download
# download('thai2fit_wv')

In [8]:
# load pretrained model
# model_pretrained = gensim.models.Word2Vec.load("/content/drive/MyDrive/Thai NLP sentiment analysis/pretrained_Model/TNCc5model.bin")

from pythainlp import word_vector
model_pretrained_pythainlp= word_vector.WordVector(model_name="thai2fit_wv").get_model() # load thai2fit_wv from pythainlp
print(len(model_pretrained_pythainlp.key_to_index))

Corpus: thai2fit_wv
- Downloading: thai2fit_wv 0.1


  0%|          | 0/62452646 [00:00<?, ?it/s]

51358


In [ ]:
# # Train the word2vec model
# # w2v_model = gensim.models.Word2Vec(X_train,
# #                                    vector_size=100,
# #                                    window=5,
# #                                    min_count=2)

# # train โมเดล
# model_pretrained.train(corpus_iterable=messages['text_clean'],total_examples=len(messages['text_clean']),epochs=300)

# model_pretrained.save("/content/drive/MyDrive/Thai NLP sentiment analysis/pretrained_Model/word2vec_pon_e300.model")


In [ ]:
# model_pretrained_pythainlp = gensim.models.Word2Vec.load("/content/drive/MyDrive/Thai NLP sentiment analysis/pretrained_Model/word2vec_pon_e300.model")
# len(model_pretrained_pythainlp.wv.index_to_key)

In [ ]:
# model_pretrained_pythainlp.wv.most_similar('ทำไม')

In [9]:
w2v_model = model_pretrained_pythainlp
print(len(w2v_model.index_to_key))
# print(len(w2v_model.wv.index_to_key))
w2v_model.most_similar('ทำไม')

51358


[('เหตุใด', 0.5505176186561584),
 ('ยังไง', 0.4701633155345917),
 ('"""..."', 0.461039274930954),
 ('เมื่อไร', 0.44824904203414917),
 ('พวกเรา', 0.4428262412548065),
 ('หรือเปล่า', 0.4396616220474243),
 ('"""“"', 0.4340546131134033),
 ('ควรจะ', 0.42570000886917114),
 ('ถ้า', 0.4255577027797699),
 ('ล่ะ', 0.42101994156837463)]

In [13]:
words = set(w2v_model.index_to_key )
X_train_vect = np.array([np.array([w2v_model[i] for i in ls if i in words])
                         for ls in messages['Text']])

# X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in messages['text_clean']])

print(X_train_vect.shape) #numpy.ndarray((1,300))

(23291,)


<ipython-input-13-de8bfcb6ec51>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model[i] for i in ls if i in words])


In [14]:
X_train_vect[0].shape

(49, 300)

In [15]:
X_train_vect[1].shape

(2, 300)

In [16]:
X_train_vect[5].shape

(426, 300)

In [17]:
X_train_vect[1].shape

(2, 300)

In [19]:
X_train_vect_avg = []
for i, v in enumerate(X_train_vect):
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
        # X_train_vect_avg.append(v[i])
    else:
        X_train_vect_avg.append(np.zeros(300, dtype=float))
        # X_train_vect_avg.append(np.zeros(100, dtype=float))

# # X_test_vect_avg = []
# # for v in X_test_vect:
# #     if v.size:
# #         X_test_vect_avg.append(v.mean(axis=0))
# #     else:
# #         X_test_vect_avg.append(np.zeros(100, dtype=float))


In [20]:
len(X_train_vect_avg)

23291

In [23]:
# # Are our sentence vector lengths consistent?
# for i, v in enumerate(X_train_vect_avg):
#     print(len(v))

In [24]:
X_train_vect_avg[224].shape

(300,)

In [25]:
X_train_vect_avg[0].shape

(300,)

In [30]:
X_train_ML.shape[0]

18632

In [ ]:
#------------ Train Model Classification ------------#

In [31]:
#LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

In [ ]:
# X_train:  (70, 21, 63)#(จำนวน, vector, chanel)
# y_train:  (70, 20) #(จำนวน, Class onthot)


In [33]:
len(X_train_vect_avg)

23291

In [32]:
# messages['text_clean']
# Encoding the label column df_Test['Class']=df_Test['Class'].map({'negative':1,'neutral':0, 'positive':2})
messages['Class onehot']=messages['Class'].map({'neutral':0, 'negative':1, 'positive':2})

YY = np.array(messages['Class onehot'])
NUM_CLASSES = len(np.unique(YY))
print('NUM_CLASSES: ', NUM_CLASSES)

YY = YY.reshape((YY.shape[0], 1)) # 100, 21, 63

Y = to_categorical(YY).astype(int)
print(Y.shape)
print(Y)

NUM_CLASSES:  3
(23291, 3)
[[1 0 0]
 [1 0 0]
 [0 1 0]
 ...
 [0 1 0]
 [1 0 0]
 [1 0 0]]


In [34]:
train_X_lstm = np.array([i for i in X_train_vect_avg])
train_X_lstm = train_X_lstm.reshape((train_X_lstm.shape[0], 300, 1))
# train_X_lstm = train_X_lstm.reshape(train_X_lstm.shape[0], 100, 1)
print(train_X_lstm.shape)

print('----------------')

X = np.array(train_X_lstm[:, :,:]) # dtype='float32'
print(X.shape)

(23291, 300, 1)
----------------
(23291, 300, 1)


In [35]:
X

array([[[-0.19204615],
        [ 0.20037268],
        [ 0.08007955],
        ...,
        [-0.09868602],
        [-0.29818079],
        [ 0.04921239]],

       [[-0.22666749],
        [-0.068755  ],
        [-0.13357449],
        ...,
        [-0.1840875 ],
        [ 0.26763099],
        [ 0.102934  ]],

       [[-0.30418184],
        [ 0.33955178],
        [ 0.09847207],
        ...,
        [-0.11749815],
        [-0.53023058],
        [ 0.12481114]],

       ...,

       [[-0.33946034],
        [ 0.14351612],
        [ 0.07878929],
        ...,
        [-0.09545075],
        [-0.20485944],
        [-0.04051704]],

       [[-0.10042466],
        [ 0.11158328],
        [ 0.1686568 ],
        ...,
        [-0.02662267],
        [-0.23808691],
        [ 0.10474196]],

       [[-0.18200642],
        [ 0.10171556],
        [ 0.14552517],
        ...,
        [-0.07859547],
        [-0.20393182],
        [-0.01486761]]])

In [ ]:
# messages['text_clean']

In [36]:
# Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split (messages['text_clean'], messages['Class'] , test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, ) #random_state=3
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)

print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

X_train:  (16303, 300, 1)
y_train:  (16303, 3)
X_test:  (6988, 300, 1)
y_test:  (6988, 3)


In [37]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Conv2D,MaxPooling2D,Flatten,Reshape,Dropout, Embedding, SpatialDropout1D
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, Bidirectional
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization

from matplotlib import pyplot
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [38]:
X_train.shape[1]

300

In [39]:
Class_label = 3
model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='tanh', input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(32, activation='relu'))
# model.add(LSTM(128, return_sequences=True, activation='tanh', input_shape=(X_train.shape[1],X_train.shape[2])))
# model.add(Bidirectional(LSTM(64)))
# model.add(Dense(32, activation='relu'))

model.add(Dense(Class_label, activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# model.compile(optimizer='Adam', loss='categorical_crossentropy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 300, 128)          66560     
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 169,603
Trainable params: 169,603
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Class_label = 3
# model = Sequential()

# model.add(LSTM(128, return_sequences=False, activation='tanh', input_shape=(X_train.shape[1],X_train.shape[2])))

# # model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(X_train.shape[1],X_train.shape[2])))
# # model.add(LSTM(128, return_sequences=True, activation='tanh'))
# # model.add(LSTM(64, return_sequences=False, activation='tanh'))
# # model.add(Dense(128, activation='relu'))
# model.add(Dense(32, activation='relu'))

# model.add(Dense(Class_label, activation='softmax'))

# model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# # model.compile(optimizer='Adam', loss='categorical_crossentropy')

# model.summary()

In [ ]:
# # res = [.7, 0.2, 0.1]
# Class_label = 3
# model = Sequential()

# model.add(LSTM(128, return_sequences=True, activation='tanh', input_shape=(X_train.shape[1],X_train.shape[2])))
# model.add(LSTM(64, return_sequences=False, activation='tanh'))
# model.add(Dense(32, activation='relu'))

# model.add(Dense(Class_label, activation='softmax'))

# model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# # model.compile(optimizer='Adam', loss='categorical_crossentropy')

# model.summary()

In [40]:
import math
batch_size = 32
n_batches = len(X_train) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer
# 5*n_batches

In [ ]:
# Load Check point

In [41]:
# filepath_checkpoint = '/content/drive/MyDrive/Thai NLP sentiment analysis/Checkpoint_LSTM3/cp-0010.ckpt'
# model.load_weights(filepath_checkpoint)
# model

In [42]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "/content/drive/MyDrive/Thai NLP sentiment analysis/Checkpoint_LSTM3/cp-{epoch:04d}.ckpt"
# checkpoint_path = "/content/drive/MyDrive/Thai NLP sentiment analysis/checkpoint_LSTM2/cp-{epoch:04d}.ckpt"


# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*n_batches)

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

In [43]:
# train โมเดล และเก็บ log
history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, callbacks=[cp_callback],
                    validation_data=(X_test, y_test), verbose=2, shuffle=False)

model.save('/content/drive/MyDrive/Thai NLP sentiment analysis/Checkpoint_LSTM3/LSTM_PON300_e50.h5')

Epoch 1/50
510/510 - 560s - loss: 0.9925 - categorical_accuracy: 0.5532 - val_loss: 0.9815 - val_categorical_accuracy: 0.5581 - 560s/epoch - 1s/step
Epoch 2/50
510/510 - 544s - loss: 0.9720 - categorical_accuracy: 0.5532 - val_loss: 0.9488 - val_categorical_accuracy: 0.5584 - 544s/epoch - 1s/step
Epoch 3/50
510/510 - 542s - loss: 0.9556 - categorical_accuracy: 0.5539 - val_loss: 0.9418 - val_categorical_accuracy: 0.5595 - 542s/epoch - 1s/step
Epoch 4/50
510/510 - 522s - loss: 0.9501 - categorical_accuracy: 0.5535 - val_loss: 0.9373 - val_categorical_accuracy: 0.5610 - 522s/epoch - 1s/step
Epoch 5/50

Epoch 5: saving model to /content/drive/MyDrive/Thai NLP sentiment analysis/Checkpoint_LSTM3/cp-0005.ckpt
510/510 - 545s - loss: 0.9452 - categorical_accuracy: 0.5557 - val_loss: 0.9313 - val_categorical_accuracy: 0.5612 - 545s/epoch - 1s/step
Epoch 6/50
510/510 - 546s - loss: 0.9396 - categorical_accuracy: 0.5571 - val_loss: 0.9224 - val_categorical_accuracy: 0.5604 - 546s/epoch - 1s/step

KeyboardInterrupt: ignored

In [44]:
filepath_checkpoint = '/content/drive/MyDrive/Thai NLP sentiment analysis/Checkpoint_LSTM3/cp-0015.ckpt'
model.load_weights(filepath_checkpoint)
# model
model.save('/content/drive/MyDrive/Thai NLP sentiment analysis/Checkpoint_LSTM3/LSTM_PON300_e15.h5')

In [ ]:
# model.save('/content/drive/MyDrive/Thai NLP sentiment analysis/LSTM_PON_95.h5')

In [ ]:
# พล็อตค่า train&test loss
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
pyplot.plot(history.history['loss'], label='train_loss')
pyplot.plot(history.history['categorical_accuracy'], label='categorical_accuracy')
pyplot.legend()
pyplot.show()

In [ ]:
pyplot.plot(history.history['val_categorical_accuracy'], label='val_acc')
pyplot.plot(history.history['categorical_accuracy'], label='train_accuracy')
pyplot.legend()
pyplot.show()

In [ ]:
# model.save('/content/drive/MyDrive/Thai NLP sentiment analysis/Checkpoint_LSTM3/LSTM_PON300_e100.h5')

In [51]:
## ------------- TEST Model ------------- ##

In [45]:
model_LSTM_PON = load_model('/content/drive/MyDrive/Thai NLP sentiment analysis/Checkpoint_LSTM3/LSTM_PON300_e15.h5')

In [46]:
yhat = model_LSTM_PON.predict(X_test)

219/219 [==============================] - 69s 311ms/step


In [ ]:
# yhat[0]

In [ ]:
# messages['Class onthot'][0]

In [ ]:
# y_test[0]

In [ ]:
# y_test.shape

In [47]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

# multilabel_confusion_matrix(ytrue, yhat)
accuracy_score(ytrue, yhat)

0.588723526044648

In [48]:
for i, value_ythat in enumerate(yhat):
  if value_ythat != ytrue[i]:
    print('Predic: ', value_ythat)
    print('ytrue: ', ytrue[i])
    print(messages['Text'][i])
    print('------------')

Streaming output truncated to the last 5000 lines.
Predic:  0
ytrue:  1
อีก เรื่อง น่าสนใจ ของ ไต้หวัน คือ ใบเสร็จ ทุก ใบ จะ เป็น หวย รัฐบาล โดยอัตโนมัติ คือ จะ แร น ด้อม เลข หลัก ติด มา ให้ หวย ออก เดือน ครั้ง รางวัล มี ตั้งแต่ ไม่ กี่ ร้อย ถึง หลาย ล้าน เหรียญ หวย ใบเสร็จ มี มา ตั้งแต่ ปี เป็นการ กึ่ง บังคับ ให้ ร้านค้า ต้อง บันทึก รายได้ เข้า ระบบ รัฐ ตรวจสอบ ได้ ป้องกัน เลี่ยง ภาษี
------------
Predic:  1
ytrue:  2
ผม ไม่ ใช้ ครับ มาสด้า
------------
Predic:  1
ytrue:  0
หมด โปรโมชั่น แล้ว ค่ะ คุณ ลูกค้า แวะ มา ลอง กัน นะคะ
------------
Predic:  0
ytrue:  2
เคย สั่ง ละ ไม่ เยอะ เท่า ใน จาน นี้ เลย น้อย มาก จริง
------------
Predic:  0
ytrue:  2
ดูด มาก น้ำท่วม ปลอด
------------
Predic:  0
ytrue:  1
หิว
------------
Predic:  0
ytrue:  1
นิสสัน
------------
Predic:  0
ytrue:  2
มี ใคร เอียน ไท อิน มั่ง คะ
------------
Predic:  0
ytrue:  2
ชอบ บ แต่ เวลา สั่งการ ด้วย เสียง ต้อง เหมือน ทะเลาะ กับ รถ แน่ เลย ย แบบ เอ็ม จี เปิด แอร์ ร้อง เสียงดัง
------------
Predic:  0
ytrue:  1
ชอบ มาก

In [49]:
for i, value_ythat in enumerate(yhat):
  if value_ythat == ytrue[i]:
    print('Predic: ', value_ythat)
    print('ytrue: ', ytrue[i])
    print(messages['Text'][i])
    print('------------')

Streaming output truncated to the last 5000 lines.
Predic:  1
ytrue:  1
และ ทรง เดียวกัน ครับ
------------
Predic:  0
ytrue:  0
วันนี้ ไม่ ออก หรอก ขี้เกียจ
------------
Predic:  0
ytrue:  0
เลี้ยง
------------
Predic:  0
ytrue:  0
แม่ ง ถาม กิน ไร ป้า ว เลย บอก ไม่ กิน ลีโอ พี่ แก บอก มี ช้าง ละ ไป เอา มา ให้
------------
Predic:  0
ytrue:  0
มี อ เม ร่า มั๊ย ครับ
------------
Predic:  0
ytrue:  0
รพ กรุงเทพ แพ้ คดี ศาล สั่ง เปิด คลับ ง ไลฟ์ พ ริ วิ เลจง คืน สิทธิ์ สมาชิก
------------
Predic:  0
ytrue:  0
อันนี้ เลย
------------
Predic:  0
ytrue:  0
เสียใจ จจจ ลิป หาย ไป ไหน ไม่ รู้ หาไม่ เจอ ออ ออ
------------
Predic:  0
ytrue:  0
ด้านหน้า เหมือน เอา ซีวิค ย่อส่วน ด้าน ท้าย เอา แอคคอร์ด มา ย่อส่วน แล้ว มา ปั่น รวมกัน ตู้ ม กลายเป็น บ ริ โอ้ อ เม ซ
------------
Predic:  0
ytrue:  0
แม่ หญิง ใช้ อาย ไลน์ เนอร์ มิสทีน ซุปเปอร์ แบล็ค ฟิคซ์ ไลน์ เนอร์ เจ้าค่ะ เขา ว่า เขียน ง่าย แห้ง เร็ว หนา
------------
Predic:  0
ytrue:  0
เอ้า คิด ว่า จบ
------------
Predic:  2
ytrue:  2
ถาม ก่อน ว่า ก

In [ ]:
# model_pretrained = gensim.models.Word2Vec.load("/content/drive/MyDrive/Thai NLP sentiment analysis/pretrained_Model/word2vec_pon_e300.model")
# # model_pretrained.wv.most_similar('ทำไม')

In [50]:
# from pythainlp import word_vector
model_pretrained_pythainlp= word_vector.WordVector(model_name="thai2fit_wv").get_model() # load thai2fit_wv from pythainlp
print(len(model_pretrained_pythainlp.key_to_index))

51358


In [ ]:
# train_X_lstm = np.array([i for i in X_train_vect_avg])
# train_X_lstm = train_X_lstm.reshape((train_X_lstm.shape[0], 300, 1))
# # train_X_lstm = train_X_lstm.reshape(train_X_lstm.shape[0], 100, 1)
# print(train_X_lstm.shape)

# print('----------------')

# X = np.array(train_X_lstm[:, :,:]) # dtype='float32'
# print(X.shape)

In [ ]:
# for ls in X_test_text:
#   print(ls)

In [ ]:
# X_test_text = np.array(('สวัสดีค่ะ ยินดีตอนรับ','ทดสอบระบบ หนึ่งสอง','กูท้อมาก ทดทรมาน','โง่มาก หดหู่ฉิบหาย เหี้ยที่สุด',
#  'เออไม่กินแล้ว พอแล้ว','ขอบคุณจ้า ทำดีมาก',''))

# X_test_text.shape

In [ ]:
# X_test_text[0]

In [52]:
X_test_text = ['สวัสดี','ทดสอบ','ทดทรมาน','โง่มาก', 'กูท้อมาก', 'หดหู่ฉิบหาย','เหี้ยที่สุด','เออไม่กินแล้ว','ขอบคุณจ้า','ทำดีมาก']
# X_test_text = ['สวัสดี','ทดสอบ','ทดทรมาน','โง่มาก', 'กูท้อมาก','ทำดีมาก']



# articles = []
# for text in X_test_text:
#     ar = []
#     for token in text.split(" "): # แบ่งคำด้วยเครื่องหมาย "|"
#         if ("www" in token.lower()):
#             if(len(ar)>0):
#                 articles.append(ar)
#             ar =[]
#         else:
#             ar.append(token)
#     articles.append(ar)

# print(articles)
# X_test_text = articles
# print('X_test_text: ', X_test_text)

words_predic = set(model_pretrained_pythainlp.index_to_key)
X_train_vect = np.array([np.array([model_pretrained_pythainlp[i] for i in ls if i in words_predic])
                         for ls in X_test_text])

print(X_train_vect.shape)

X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        # X_train_vect_avg.append(np.zeros(100, dtype=float))
        X_train_vect_avg.append(np.zeros(300, dtype=float))


train_X_lstm_predic = np.array([i for i in X_train_vect_avg])
train_X_lstm_predic = train_X_lstm_predic.reshape((train_X_lstm_predic.shape[0], 300, 1))
# train_X_lstm_predic = train_X_lstm_predic.reshape(train_X_lstm_predic.shape[0], 100, 1)
print(train_X_lstm_predic.shape)

print('----------------')

X_prdic = np.array(train_X_lstm_predic[:, :,:]) # dtype='float32'
print(X_prdic.shape)

(10,)
(10, 300, 1)
----------------
(10, 300, 1)


<ipython-input-52-0670eec904ac>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([model_pretrained_pythainlp[i] for i in ls if i in words_predic])


In [ ]:
X_test_text[0]

In [ ]:
X_prdic[0].shape

In [53]:
# {'negative':1,'neutral':0, 'positive':2}

dic_label = {0:'neutral', 1:'negative', 2:'positive'}


yhat_P = model_LSTM_PON.predict(X_prdic)

print(yhat_P)
for i, Value in enumerate(yhat_P):
  print(X_test_text[i])
  print(np.argmax(Value))
  print('Predicted value is =', dic_label[np.argmax(Value)])
  print('--------')


1/1 [==============================] - 0s 147ms/step
[[0.7579798  0.01811137 0.22390877]
 [0.5539072  0.11710937 0.32898352]
 [0.6039359  0.15560809 0.24045609]
 [0.36659014 0.4648532  0.1685566 ]
 [0.48319727 0.05338773 0.46341506]
 [0.7306561  0.0813004  0.18804342]
 [0.69497997 0.08035101 0.22466895]
 [0.4787567  0.3135052  0.20773809]
 [0.45919314 0.22362223 0.3171846 ]
 [0.25899988 0.02391173 0.7170884 ]]
สวัสดี
0
Predicted value is = neutral
--------
ทดสอบ
0
Predicted value is = neutral
--------
ทดทรมาน
0
Predicted value is = neutral
--------
โง่มาก
1
Predicted value is = negative
--------
กูท้อมาก
0
Predicted value is = neutral
--------
หดหู่ฉิบหาย
0
Predicted value is = neutral
--------
เหี้ยที่สุด
0
Predicted value is = neutral
--------
เออไม่กินแล้ว
0
Predicted value is = neutral
--------
ขอบคุณจ้า
0
Predicted value is = neutral
--------
ทำดีมาก
2
Predicted value is = positive
--------


In [ ]:
## ------------- X_test = Test Set ไม่เคยเห็นมาก่อน ------------- ##

In [54]:
messages_Test = pd.read_csv('/content/drive/MyDrive/Thai NLP sentiment analysis/Dataset/Test01_Clear.csv')# [:5000]
messages_Test

,Unnamed: 0,Class,Text
0,0,negative,ซื้อ แต่ ผ้าอนามัย แบบ เย็น มา ค่ะ แบบ ว่า อี ...
1,1,neutral,ครับ
2,2,negative,การ ด่า ไป เหมือน ได้ บรรเทา ความเครียด เฉย แต...
3,3,neutral,ขวด
4,4,neutral,ทาน ได้ ค่ะ น้ำ ซุป ต้ม มาจาก หัวผักกาด ซีอิ้ว...
...,...,...,...
2584,2669,positive,เล็ก ลงมา แต่ ดี เท่าเดิม แวะ มาหา ได้ที่ เซเว...
2585,2670,neutral,แพนด้า ซิ่ง กับ มุมมอง ที่ แตกต่าง อย่าง ลงตัว...
2586,2671,positive,ไป เถอะ ดอม
2587,2672,neutral,เจมส์ บอนด์ ขับ โตโยต้า อัลติส


In [55]:
messages_Test['Class onthot']=messages_Test['Class'].map({'neutral':0, 'negative':1, 'positive':2})

YY_Test = np.array(messages_Test['Class onthot'])
NUM_CLASSES = len(np.unique(YY_Test))
print('NUM_CLASSES: ', NUM_CLASSES)

# YY = YY.reshape((YY.shape[0], 1)) # 100, 21, 63

YTest = to_categorical(YY_Test).astype(int)
print(YTest.shape)
print(YTest)

NUM_CLASSES:  3
(2589, 3)
[[0 1 0]
 [1 0 0]
 [0 1 0]
 ...
 [0 0 1]
 [1 0 0]
 [1 0 0]]


In [56]:
XTest = messages_Test['Text'].tolist()
XTest[:5]

['ซื้อ แต่ ผ้าอนามัย แบบ เย็น มา ค่ะ แบบ ว่า อี ห่า กู นอน ไม่ ได้',
 'ครับ',
 'การ ด่า ไป เหมือน ได้ บรรเทา ความเครียด เฉย แต่ บีทีเอส รถไฟฟ้า มัน สำนึก มั้ย ก็ ไม่ อ่ะ',
 'ขวด',
 'ทาน ได้ ค่ะ น้ำ ซุป ต้ม มาจาก หัวผักกาด ซีอิ้ว ขาว เกลือ แล น้ำตาล ค่ะ']

In [58]:
X_test_text = XTest

words_predic = set(model_pretrained_pythainlp.index_to_key)
X_train_vect = np.array([np.array([model_pretrained_pythainlp[i] for i in ls if i in words_predic])
                         for ls in X_test_text])

print(X_train_vect.shape)

X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        # X_train_vect_avg.append(np.zeros(100, dtype=float))
        X_train_vect_avg.append(np.zeros(300, dtype=float))


train_X_lstm_predic = np.array([i for i in X_train_vect_avg])
train_X_lstm_predic = train_X_lstm_predic.reshape((train_X_lstm_predic.shape[0], 300, 1))
# train_X_lstm_predic = train_X_lstm_predic.reshape(train_X_lstm_predic.shape[0], 100, 1)
print(train_X_lstm_predic.shape)

print('----------------')

X_prdic = np.array(train_X_lstm_predic[:, :,:]) # dtype='float32'
print(X_prdic.shape)

(2589,)
(2589, 300, 1)
----------------
(2589, 300, 1)


<ipython-input-58-5bb0e61739cd>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([model_pretrained_pythainlp[i] for i in ls if i in words_predic])


In [60]:
yTest_hat = model_LSTM_PON.predict(X_prdic)

yTest_true = np.argmax(YTest, axis=1).tolist()
yTest_hat = np.argmax(yTest_hat, axis=1).tolist()

# multilabel_confusion_matrix(ytrue, yhat)
accuracy_score(yTest_true, yTest_hat)

81/81 [==============================] - 27s 329ms/step


0.5983005021243724